In [17]:
import datetime
from datetime import timedelta
import numpy as np
import os
import pandas as pd
import time
import yahoo_fin.stock_info as si

import logging

handler_for_errors = logging.FileHandler("errors.log", mode='w')
handler_for_info = logging.FileHandler("logging_de.log", mode='w')
handler_for_errors.setFormatter(logging.Formatter(fmt = "%(asctime)s %(name)s %(levelname)s %(message)s", \
                                                      datefmt = "%Y-%m-%d %H:%M:%S")) 
handler_for_info.setFormatter(logging.Formatter(fmt = "%(asctime)s %(name)s %(levelname)s %(message)s", \
                                                      datefmt = "%Y-%m-%d %H:%M:%S"))
class ErrorFilter(logging.Filter):
    def filter(self, record):
        return record.levelno >= logging.ERROR
handler_for_errors.addFilter(ErrorFilter())

class InfoWarningFilter(logging.Filter):
    def filter(self, record):
        return record.levelno <= logging.WARNING
handler_for_info.addFilter(InfoWarningFilter())

    
logger = logging.getLogger("logger")
logger.setLevel(logging.DEBUG)
logger.addHandler(handler_for_errors)
logger.addHandler(handler_for_info)

class Parser:
    def __init__(self, stock_file_name_constructor, requests_filename,\
                 stock_df_header, first_request_history_days = 1095):
        self.request_id = 0
        logger.info("Start initializing parser")
        self.last_request_dict = {}
        
        self.stock_file_name_constructor = stock_file_name_constructor
        try:
            stock_file_name_constructor("AAPL")
        except:
            logger.error("Parser is given incorrect function")
            
        self.requests_filename = requests_filename
        self.stock_df_header = stock_df_header
        self.update_threshold = datetime.timedelta(days = 1)
        self.first_request_history_days = first_request_history_days
        logger.debug("Parser initialized successfully")
        
    def update_history(self):
        self.request_id += 1
        current_date = datetime.datetime.now()
        logger.info(f'Request {self.request_id} has started at {current_date}')
        if (os.path.isfile(self.requests_filename)):
            logger.debug(f'Requests file found')
            requests = open(self.requests_filename, "r")
            requests_heading = requests.readline()
            logger.debug('Requests file opened')
            for raw_row in requests:
                try:
                    logger.debug("Detecting stock name")
                    row = raw_row.split(",")
                    stock_name = row[0]
                    logger.debug(f'Stock name {stock_name} detected')
                    stock_file_name = self.stock_file_name_constructor(stock_name)
                    if stock_name in self.last_request_dict:
                        logger.debug(f'{stock_name} has been updated {self.last_request_dict[stock_name]} last time')
                        start_date = self.last_request_dict[stock_name]
                    else:
                        logger.debug(f'No history for {stock_name} can be found. Find prices for the last {self.first_request_history_days} days')
                        start_date = current_date - datetime.timedelta(days=self.first_request_history_days)
                        stock_file = open(stock_file_name, "w")
                        stock_file.write(self.stock_df_header)
                        stock_file.close()
                    if (current_date - start_date > self.update_threshold):
                        logger.debug(f'New information is likely to appear. Start updating {stock_name} data')
                        historical_prices = si.get_data(stock_name, start_date, current_date)
                        historical_prices.to_csv(path_or_buf = stock_file_name, header = False, mode = 'a')
                        self.last_request_dict[stock_name] = current_date
                        logger.debug(f'Information has been succesfully updated')
                    else:
                        logger.debug(f'{stock_name} data has been recently updated at {self.last_request_dict[stock_name]}. No new information has appeared')
                except Exception as e:
                    logger.error(f'Error {e} has occured')
            requests.close()
            os.remove(self.requests_filename)
            logger.debug(f'Requests file has been deleted')
        else:
            logger.debug(f'No requests file found')
        logger.info(f'Request {self.request_id} has finished at {datetime.datetime.now()}')
            

def history_stock_df_name(stock_name):
    return "history_" + stock_name + ".csv"

requests_filename = "current_requests.csv"
stock_df_header = "Datetime,Open,High,Low,Close,Adj Close,Volume,Ticker\n"

default_parser = Parser(stock_file_name_constructor = history_stock_df_name,\
                        requests_filename = requests_filename,\
                        stock_df_header = stock_df_header)


In [20]:
default_parser.update_history()

In [3]:
import datetime
import yahoo_fin.stock_info as si

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=14)
historical_prices = si.get_data("EUR=X", start_date, end_date)

historical_prices

,open,high,low,close,adjclose,volume,ticker
2023-12-11,0.92887,0.93082,0.927700,0.92887,0.92887,0,EUR=X
2023-12-12,0.92880,0.92921,0.924040,0.92880,0.92880,0,EUR=X
2023-12-13,0.92604,0.92816,0.925220,0.92604,0.92604,0,EUR=X
2023-12-14,0.91860,0.91910,0.908750,0.91860,0.91860,0,EUR=X
2023-12-15,0.90962,0.91724,0.908600,0.90962,0.90962,0,EUR=X
2023-12-18,0.91783,0.91789,0.914910,0.91783,0.91783,0,EUR=X
2023-12-19,0.91543,0.91608,0.910084,0.91543,0.91543,0,EUR=X
2023-12-20,0.91066,0.91444,0.910770,0.91066,0.91066,0,EUR=X
2023-12-21,0.91360,0.91439,0.909110,0.91360,0.91360,0,EUR=X
2023-12-22,0.90858,0.90957,0.905750,0.90858,0.90858,0,EUR=X
